In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
import pandas as pd
data = pd.read_csv("C:/Users/chukk/Desktop/fake_news.csv") 
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
data.shape

(20800, 5)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
data=data.drop(['id'],axis=1)

In [9]:
data=data.fillna('')

In [10]:
data['content']=data['author']+' '+data['title']+' '+data['text']

In [11]:
data=data.drop(['title','author','text'],axis=1)

In [12]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop=stopwords.words('english')
data['content']=data['content'].apply(lambda x:" ".join([word for word in x.split() if word not in stop]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chukk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
!pip install textblob


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content']=data['content'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chukk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why Truth Might Get You Fir...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed fictional ...
Name: content, dtype: object

In [15]:
X=data[['content']]
y=data['label']

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=45,stratify=y)

In [18]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14560, 1)
(14560,)
(6240, 1)
(6240,)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer


hi


In [ ]:
tfidf_vect=TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=5000)
tfidf_vect.fit(data['content'])
x_train_tfidf=tfidf_vect.transform(X_train['content'])
xtest_tfidf=tfidf_vect.transform(X_test['content'])

In [34]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf=PassiveAggressiveClassifier()
pclf.fit(x_train_tfidf,y_train)
predictions=pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3116
           1       0.97      0.97      0.97      3124

    accuracy                           0.97      6240
   macro avg       0.97      0.97      0.97      6240
weighted avg       0.97      0.97      0.97      6240



In [35]:
print(metrics.confusion_matrix(y_test,predictions))

[[3017   99]
 [ 105 3019]]


In [36]:
from sklearn.neural_network import MLPClassifier
mlpclf=MLPClassifier(hidden_layer_sizes=(256,64,16),
                    activation='relu',
                    solver='adam')
mlpclf.fit(x_train_tfidf,y_train)
predictions=mlpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

C:\Users\chukk\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3116
           1       0.97      0.96      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [24]:
print(metrics.confusion_matrix(y_test,predictions))

[[3014  102]
 [ 118 3006]]
